In [1]:
text = "This is some text."
byte_array = bytearray(text, 'utf-8')
print(byte_array)
print(list(byte_array))  # each integer corresponds to the byte value

bytearray(b'This is some text.')
[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116, 46]


In [2]:
# byte = 8 bits = 256 possible values, 0 to 255
print(bytearray(range(0, 257)))  # gives error

ValueError: byte must be in range(0, 256)

In [3]:
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding('gpt2')
# bpe uses 0-255 to represent its first 256 single-character tokens
for i in range(300):
    print(f'{i:03}: {gpt2_tokenizer.decode([i])}')

000: !
001: "
002: #
003: $
004: %
005: &
006: '
007: (
008: )
009: *
010: +
011: ,
012: -
013: .
014: /
015: 0
016: 1
017: 2
018: 3
019: 4
020: 5
021: 6
022: 7
023: 8
024: 9
025: :
026: ;
027: <
028: =
029: >
030: ?
031: @
032: A
033: B
034: C
035: D
036: E
037: F
038: G
039: H
040: I
041: J
042: K
043: L
044: M
045: N
046: O
047: P
048: Q
049: R
050: S
051: T
052: U
053: V
054: W
055: X
056: Y
057: Z
058: [
059: \
060: ]
061: ^
062: _
063: `
064: a
065: b
066: c
067: d
068: e
069: f
070: g
071: h
072: i
073: j
074: k
075: l
076: m
077: n
078: o
079: p
080: q
081: r
082: s
083: t
084: u
085: v
086: w
087: x
088: y
089: z
090: {
091: |
092: }
093: ~
094: �
095: �
096: �
097: �
098: �
099: �
100: �
101: �
102: �
103: �
104: �
105: �
106: �
107: �
108: �
109: �
110: �
111: �
112: �
113: �
114: �
115: �
116: �
117: �
118: �
119: �
120: �
121: �
122: �
123: �
124: �
125: �
126: �
127: �
128: �
129: �
130: �
131: �
132: �
133: �
134: �
135: �
136: �
137: �
138: �
139: �
140: �
141: �
142: �

### Idea
- Identify frequent pairs of bytes
- Replace pair with a new placeholder ID, e.g. 256 if we start with 0-255
- Record in a lookup table, the size of the table is the vocabulary size
- Repeat
- Stop when no pair occurs more than once

In [22]:
from collections import Counter, deque
from functools import lru_cache
import json

class BPETokenizerSimple:
    def __init__(self):
        self.vocab = {}  # ID -> token
        self.inv_vocab = {}  # token -> ID
        self.bpe_merges = {}  # e.g: {(id1, id2): merged_id}
        # for OpenAI GPT-2 merges, e.g: {(str1, str2): rank}
        # lower rank = higher priority
        self.bpe_ranks = {}
    
    def train(self, text, vocab_size, allowed_special={'<|endoftext|>'}):
        # replace space with Ġ (particularity of GPT-2 BPE)
        # e.g: "Hello world" -> ["Hello", "Ġworld"]
        # GPT-4 BPE would give ["Hello", " world"]
        preproc_text = []
        for i, ch in enumerate(text):
            if ch == ' ' and i != 0:
                preproc_text.append('Ġ')
            if ch != ' ':
                preproc_text.append(ch)
        preproc_text = ''.join(preproc_text)

        # start with first 0-255 ASCII
        unique_chrs = [chr(i) for i in range(256)]
        # add other chars, like Ġ
        unique_chrs.extend(ch for ch in sorted(set(preproc_text)) if ch not in unique_chrs)
        if 'Ġ' not in unique_chrs:
            unique_chrs.append('Ġ')
        
        self.vocab = {i: ch for i, ch in enumerate(unique_chrs)}
        self.inv_vocab = {ch: i for i, ch in self.vocab.items()}

        if allowed_special:
            for token in allowed_special:
                # use N as new ID
                new_id = len(self.vocab)
                self.vocab[new_id] = token
                self.inv_vocab[token] = new_id
        
        # tokenize
        token_ids = [self.inv_vocab[ch] for ch in preproc_text]

        # BPE
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode='most')
            
            # stop if no pair occurs more than once
            if pair_id is None:
                break
            
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id
        
        # add to vocabulary
        for (p0, p1), new_id in self.bpe_merges.items():
            merged = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged
            self.inv_vocab[merged] = new_id

    @staticmethod
    def find_freq_pair(token_ids, mode='most'):
        # pairs of (n, n+1)
        pairs = Counter(zip(token_ids, token_ids[1:]))
        
        if not pairs:
            return None
        
        # key is based on counts, stored in x[1]
        if mode == 'most':
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == 'least':
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError('Invalid mode. Must be "most" or "least"')
    
    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            cur = dq.popleft()
            # check if cur and cur+1 make the pair
            if dq and (cur, dq[0]) == pair_id:
                # replace cur and cur+1 with a single id
                replaced.append(new_id)
                dq.popleft()
            else:
                replaced.append(cur)
        
        return replaced

    def encode(self, text, allowed_special=None):
        import re

        token_ids = []

        if allowed_special is not None and len(allowed_special) > 0:
            # escape special characters then merge each special with |
            # creates a pattern like (special1|special2|special3|...)
            special_regex_pattern = "(" + \
            "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) \
            + ")"

            last_idx = 0
            for match in re.finditer(special_regex_pattern, text):
                # all characters before a special token
                prefix = text[last_idx:match.start()]
                # prefix requires no special handling
                token_ids.extend(self.encode(prefix, allowed_special=None))

                special_token = match.group(0)
                if special_token in self.inv_vocab:
                    token_ids.append(self.inv_vocab[special_token])
                else:
                    raise ValueError(f'Special token {special_token} not in vocabulary.')

                last_idx = match.end()
            
            # all characters after last special token
            text = text[last_idx:]

            disallowed = [
                tok for tok in self.inv_vocab
                if tok.startswith('<|') and tok.endswith('|>') \
                and tok in text and tok not in allowed_special
            ]

            if disallowed:
                raise ValueError(f'Disallowed special tokens found in text: {disallowed}')
        
        # text will be treated as having no special tokens at this point
        tokens = []
        lines =  text.split('\n')
        for i, line in enumerate(lines):
            if i > 0:  # separate each line with newline
                tokens.append('\n')
            
            words = line.split()
            for j, word in enumerate(words):
                if j == 0 and i > 0:  # first word and not first line
                    # separate words by Ġ
                    tokens.append('Ġ'+word)
                elif j == 0:  # first word and first line
                    tokens.append(word)
                else:
                    tokens.append('Ġ'+word)
        
        for token in tokens:
            if token in self.inv_vocab:
                token_ids.append(self.inv_vocab[token])
            else:
                token_ids.extend(self.tokenize_with_bpe(token))
        
        return token_ids
    
    def tokenize_with_bpe(self, token):
        # tokenize into individual characters
        token_ids = [self.inv_vocab.get(ch, None) for ch in token]
        
        if None in token_ids:
            missing_chars = [ch for ch, tok_id in zip(token, token_ids) if tok_id is None]
            raise ValueError(f'Not found in vocab: {missing_chars}')
        
        # if GPT-2 merges aren't loaded
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0

                # N-1 because we access i+1
                while i < len(token_ids)-1:
                    pair = (token_ids[i], token_ids[i+1])
                    
                    if pair in self.bpe_merges:
                        merged_id = self.bpe_merges[pair]
                        new_tokens.append(merged_id)
                        i += 2  # merged with next token, so skip it
                        can_merge = True
                    else:
                        # (i, i+1) is not a pair in the merges
                        new_tokens.append(token_ids[i])
                        i += 1
                
                # last token ID
                # this will be skipped if (N-1, N) was a merged pair in the previous loop
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                
                # iterate until all merges are done
                token_ids = new_tokens
            
            return token_ids
        
    def decode(self, token_ids):
        decoded = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f'Token ID {token_id} not in vocab')

            token = self.vocab[token_id]
            if token == '\n':
                if decoded and not decoded.endswith(" "):
                    decoded += " "  # add space before a newline
                decoded += token
            elif token.startswith('Ġ'):
                # Ġ represented a space in BPE
                decoded += ' ' + token[1:]
            else:
                decoded += token
        
        return decoded

In [10]:
import os

verdict_path = os.sep.join([os.getcwd(), 'deps', 'the-verdict.txt'])
with open(verdict_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [23]:
tokenizer = BPETokenizerSimple()

# default vocab size is 256 (due to 0-255 ASCII)
# GPT-2 vocab size is 50,257
# GPT-4 vocab size is 100,256
# GPT-4o vocab size is 199,997
tokenizer.train(text, vocab_size=1000, allowed_special={'<|endoftext|>'})

In [24]:
print(len(tokenizer.vocab))

1000


In [25]:
tokenizer.vocab

{0: '\x00',
 1: '\x01',
 2: '\x02',
 3: '\x03',
 4: '\x04',
 5: '\x05',
 6: '\x06',
 7: '\x07',
 8: '\x08',
 9: '\t',
 10: '\n',
 11: '\x0b',
 12: '\x0c',
 13: '\r',
 14: '\x0e',
 15: '\x0f',
 16: '\x10',
 17: '\x11',
 18: '\x12',
 19: '\x13',
 20: '\x14',
 21: '\x15',
 22: '\x16',
 23: '\x17',
 24: '\x18',
 25: '\x19',
 26: '\x1a',
 27: '\x1b',
 28: '\x1c',
 29: '\x1d',
 30: '\x1e',
 31: '\x1f',
 32: ' ',
 33: '!',
 34: '"',
 35: '#',
 36: '$',
 37: '%',
 38: '&',
 39: "'",
 40: '(',
 41: ')',
 42: '*',
 43: '+',
 44: ',',
 45: '-',
 46: '.',
 47: '/',
 48: '0',
 49: '1',
 50: '2',
 51: '3',
 52: '4',
 53: '5',
 54: '6',
 55: '7',
 56: '8',
 57: '9',
 58: ':',
 59: ';',
 60: '<',
 61: '=',
 62: '>',
 63: '?',
 64: '@',
 65: 'A',
 66: 'B',
 67: 'C',
 68: 'D',
 69: 'E',
 70: 'F',
 71: 'G',
 72: 'H',
 73: 'I',
 74: 'J',
 75: 'K',
 76: 'L',
 77: 'M',
 78: 'N',
 79: 'O',
 80: 'P',
 81: 'Q',
 82: 'R',
 83: 'S',
 84: 'T',
 85: 'U',
 86: 'V',
 87: 'W',
 88: 'X',
 89: 'Y',
 90: 'Z',
 91: '[',


In [26]:
print(len(tokenizer.bpe_merges))

742


In [27]:
tokenizer.bpe_merges

{(101, 256): 258,
 (256, 116): 259,
 (100, 256): 260,
 (116, 256): 261,
 (105, 110): 262,
 (115, 256): 263,
 (104, 258): 264,
 (104, 97): 265,
 (44, 256): 266,
 (111, 117): 267,
 (101, 114): 268,
 (97, 110): 269,
 (111, 110): 270,
 (101, 110): 271,
 (259, 264): 272,
 (121, 256): 273,
 (46, 256): 274,
 (111, 256): 275,
 (262, 103): 276,
 (104, 105): 277,
 (105, 116): 278,
 (101, 260): 279,
 (73, 256): 280,
 (111, 102): 281,
 (119, 97): 282,
 (115, 116): 283,
 (45, 45): 284,
 (114, 101): 285,
 (111, 114): 286,
 (256, 97): 287,
 (10, 10): 288,
 (97, 116): 289,
 (101, 108): 290,
 (97, 114): 291,
 (269, 260): 292,
 (117, 114): 293,
 (259, 275): 294,
 (111, 119): 295,
 (98, 101): 296,
 (281, 256): 297,
 (276, 256): 298,
 (105, 115): 299,
 (277, 263): 300,
 (97, 108): 301,
 (97, 99): 302,
 (259, 104): 303,
 (105, 99): 304,
 (111, 109): 305,
 (103, 104): 306,
 (258, 116): 307,
 (265, 261): 308,
 (282, 263): 309,
 (97, 256): 310,
 (101, 100): 311,
 (271, 256): 312,
 (104, 268): 313,
 (265, 260)

In [28]:
input_text = "Jack embraced beauty through art and life."
print(tokenizer.encode(input_text))

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [29]:
# with <|endoftext|>
input_text = "Jack embraced beauty through art and life.<|endoftext|>"
print(tokenizer.encode(input_text))

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 60, 124, 271, 683, 102, 116, 461, 116, 124, 62]


In [30]:
# with <|endoftext|>
input_text = "Jack embraced beauty through art and life.<|endoftext|>"
print(tokenizer.encode(input_text, allowed_special={'<|endoftext|>'}))

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257]


In [31]:
print(tokenizer.decode(tokenizer.encode(input_text, allowed_special={'<|endoftext|>'})))

Jack embraced beauty through art and life.<|endoftext|>
